# Transformer models

## Load data

In [1]:
from hanziconv import HanziConv
from ezra import BibleSearchEngine, BibleSearchStrategy

ezra_engine = BibleSearchEngine()
ezra_engine.bible['text_s'] = ezra_engine.bible.text.apply(HanziConv.toSimplified)

## Compute embeddings

In [2]:
import pickle
from sentence_transformers import SentenceTransformer, util

max_seq = ezra_engine.bible.text_s.str.len().max()

class SentenceTransformerStrategy(BibleSearchStrategy):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)
        self.model.max_seq_length = max_seq
        self.verse_emb = self.model.encode(ezra_engine.bible.text_s, show_progress_bar=True)
        self.save_embeddings(f'data/embeddings/{model_name}.pkl')

    def save_embeddings(self, path: str):
        with open(path, "wb") as fOut:
            pickle.dump(self.verse_emb, fOut, protocol=pickle.HIGHEST_PROTOCOL)

    def search(self, keyword, top_k):
        keyword_emb = self.model.encode([keyword])
        results = util.semantic_search(keyword_emb, self.verse_emb)[0]
        return [(r['corpus_id'], r['score']) for r in results]

In [3]:
wwm = SentenceTransformerStrategy('hfl/chinese-bert-wwm')
ezra_engine.strategy = wwm

Exception when trying to download https://sbert.net/models/hfl/chinese-bert-wwm.zip. Response 404
Batches: 100%|██████████| 972/972 [2:44:47<00:00, 10.17s/it]


In [4]:
searches = ['挂虑 祈祷', '喜乐 事奉', '求救', '信心 行事']

In [5]:
for kw in searches:
    results = ezra_engine.search(kw, verbose=True)

Searches for 挂虑 祈祷:
Score:  0.7362 我便禁食，披麻蒙灰，定意向主神祈禱懇求。
Score:  0.7213 在思念夜中、異象之間，世人沉睡的時候，
Score:  0.7213 我心中急躁，所以我的思念叫我回答。
Score:  0.7168 我想念神，就煩躁不安；我沉吟悲傷，心便發昏。（細拉）
Score:  0.7104 不住地禱告，
Score:  0.7096 像奴僕切慕黑影，像雇工人盼望工價；
Score:  0.7085 恐懼戰兢歸到我身；驚恐漫過了我。
Score:  0.7084 他當口貼塵埃，或者有指望。
Score:  0.7066 常在殿裡稱頌神。
Score:  0.7033 人因多受欺壓就哀求，因受能者的轄制（原文是膀臂）便求救，

Searches for 喜乐 事奉:
Score:  0.8281 因聽見你們在基督耶穌裡的信心，並向眾聖徒的愛心，
Score:  0.8228 為他們記念他的約，照他豐盛的慈愛後悔。
Score:  0.8216 向我們列祖施憐憫，記念他的聖約─
Score:  0.8208 我心渴想你的救恩，仰望你的應許。
Score:  0.8206 但我倚靠你的慈愛；我的心因你的救恩快樂。
Score:  0.8184 願平安、仁愛、信心從父神和主耶穌基督歸與弟兄們！
Score:  0.8170 願恩惠、平安從神我們的父並主耶穌基督歸與你們！
Score:  0.8162 要常常喜樂，
Score:  0.8148 你們要用愛心彼此親嘴問安。願平安歸與你們凡在基督裡的人！
Score:  0.8141 門徒滿心喜樂，又被聖靈充滿。

Searches for 求救:
Score:  0.7617 a
Score:  0.7617 a
Score:  0.7617 a
Score:  0.7617 a
Score:  0.7617 a
Score:  0.7617 a
Score:  0.7617 a
Score:  0.7617 a
Score:  0.7617 a
Score:  0.7617 a

Searches for 信心 行事:
Score:  0.7641 但要凡事察驗；善美的要持守，
Score:  0.7587 惡人臉無羞恥；正直人行事堅定。
Score:  0

In [ ]:
du = SentenceTransformerStrategy('distiluse-base-multilingual-cased-v2')
ezra_engine.strategy = du

In [66]:
for kw in searches:
    results = ezra_engine.search(kw, verbose=True)

Searches for 挂虑 祈祷:
Score:  0.8784 不住地禱告，
Score:  0.6724 請弟兄們為我們禱告。
Score:  0.6638 你們要恆切禱告，在此警醒感恩。
Score:  0.6236 求你從天上垂聽他們的禱告祈求，使他們得勝。
Score:  0.6216 求你在天上垂聽他們的禱告祈求，使他們得勝。
Score:  0.6107 神啊，求你聽我的禱告，留心聽我口中的言語。
Score:  0.6091 你們禱告，無論求甚麼，只要信，就必得著。」
Score:  0.6088 應當一無掛慮，只要凡事藉著禱告、祈求，和感謝，將你們所要的告訴神。
Score:  0.5979 你們要呼求我，禱告我，我就應允你們。
Score:  0.5791 聽禱告的主啊，凡有血氣的都要來就你。

Searches for 喜乐 事奉:
Score:  0.6796 要常常喜樂，
Score:  0.4753 折腳折手的、
Score:  0.4182 惟有義人必然歡喜，在神面前高興快樂。
Score:  0.4167 猶大人有光榮，歡喜快樂而得尊貴。
Score:  0.4079 說：
Score:  0.4074 卑微的弟兄升高，就該喜樂；
Score:  0.3984 以利戶又說：
Score:  0.3984 以利戶又說：
Score:  0.3937 你們要靠主常常喜樂。我再說，你們要喜樂。
Score:  0.3900 不住地禱告，

Searches for 求救:
Score:  0.4954 說：
Score:  0.4761 不住地禱告，
Score:  0.4265 拯救我的主啊，求你快快幫助我！
Score:  0.4238 亞希雅、哈難、亞難、
Score:  0.4010 折腳折手的、
Score:  0.3940 a
Score:  0.3940 a
Score:  0.3940 a
Score:  0.3940 a
Score:  0.3940 a

Searches for 信心 行事:
Score:  0.4612 可見，信心是與他的行為並行，而且信心因著行為才得成全。
Score:  0.4397 必有人說：「你有信心，我有行為；你將你沒有行為的信心指給我看，我便藉著我的行為，將我的信心指給你看。」
Sco

In [64]:
cb = SentenceTransformerStrategy('bert-base-chinese')
ezra_engine.strategy = cb

Exception when trying to download https://sbert.net/models/bert-base-chinese.zip. Response 404
Downloading: 100%|██████████| 624/624 [00:00<00:00, 165kB/s]
Downloading: 100%|██████████| 412M/412M [22:01<00:00, 312kB/s]
Downloading: 100%|██████████| 110k/110k [00:01<00:00, 107kB/s]  
Downloading: 100%|██████████| 269k/269k [00:01<00:00, 171kB/s]
Batches: 100%|██████████| 972/972 [3:09:39<00:00, 11.71s/it]


In [73]:
for kw in searches:
    results = ezra_engine.search(kw, verbose=True)

Searches for 挂虑 祈祷:
Score:  0.7046 我便禁食，披麻蒙灰，定意向主神祈禱懇求。
Score:  0.6962 我想念神，就煩躁不安；我沉吟悲傷，心便發昏。（細拉）
Score:  0.6863 每逢為你們眾人祈求的時候，常是歡歡喜喜地祈求。
Score:  0.6860 我勸你，第一要為萬人懇求、禱告、代求、祝謝；
Score:  0.6840 從何羅念有喊荒涼大毀滅的哀聲：
Score:  0.6720 「當記念安息日，守為聖日。
Score:  0.6706 我有憂愁，願能自慰；我心在我裡面發昏。
Score:  0.6700 恐懼戰兢歸到我身；驚恐漫過了我。
Score:  0.6697 在思念夜中、異象之間，世人沉睡的時候，
Score:  0.6662 有一宗人（宗：原文是代；下同），咒詛父親，不給母親祝福。

Searches for 喜乐 事奉:
Score:  0.8040 要常常喜樂，
Score:  0.7612 常在殿裡稱頌神。
Score:  0.7516 我靈以神我的救主為樂；
Score:  0.7488 總要察驗何為主所喜悅的事。
Score:  0.7441 卑微的弟兄升高，就該喜樂；
Score:  0.7429 不住地禱告，
Score:  0.7420 猶大人有光榮，歡喜快樂而得尊貴。
Score:  0.7405 散布亮光是為義人；預備喜樂是為正直人。
Score:  0.7403 所羅門因為求這事，就蒙主喜悅。
Score:  0.7373 在那裡傳福音。

Searches for 求救:
Score:  0.7817 a
Score:  0.7817 a
Score:  0.7817 a
Score:  0.7817 a
Score:  0.7817 a
Score:  0.7817 a
Score:  0.7817 a
Score:  0.7817 a
Score:  0.7817 a
Score:  0.7817 a

Searches for 信心 行事:
Score:  0.7413 但要凡事察驗；善美的要持守，
Score:  0.7335 我知道世人，莫強如終身喜樂行善；
Score:  0.7248 你所命定的法度是憑公義和至誠。
Score:  0.7236 若是能行，總要